In [ ]:
%cd /home/adityasidharta/git/shopee_data_science
%env PROJECT_PATH /home/adityasidharta/git/shopee_data_science

In [ ]:
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os
from sklearn.model_selection import train_test_split
from utils.envs import *
import string
from utils.common import create_directory

from pathlib import Path
from category_encoders import OrdinalEncoder

In [ ]:
create_directory(os.path.join(result_path, 'static'))
RESULT_PATH = Path(os.path.join(result_path, 'static'))

In [ ]:
data_lm = load_data(RESULT_PATH, 'data_lm.pkl')
learn_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)
learn_lm.load(RESULT_PATH/'model_lm.pkl')
learn_lm.load_encoder(RESULT_PATH/'encoder_lm.pkl')

In [ ]:
beauty_train = pd.read_csv(beauty_dev_repo)
beauty_test = pd.read_csv(beauty_val_repo)

fashion_train = pd.read_csv(fashion_dev_repo)
fashion_test = pd.read_csv(fashion_val_repo)

mobile_train = pd.read_csv(mobile_dev_repo)
mobile_test = pd.read_csv(mobile_val_repo)

In [ ]:
train_df = mobile_train
test_df = mobile_test
topic = 'Camera'

In [ ]:
df_trn = train_df[[topic, 'title']].copy()
df_tst = test_df[[topic, 'title']].copy()

df_trn.columns = ['label', 'text']
df_tst.columns = ['label', 'text']

df_trn = df_trn.dropna()

df_trn['label'] = df_trn['label'].astype(str)
df_tst['label'] = df_tst['label'].astype(str)

df_trn, df_val = train_test_split(df_trn, test_size = 0.2)

In [ ]:
print(train_df.shape)
print(test_df.shape)

print(df_trn.shape)
print(df_val.shape)
print(df_tst.shape)

In [ ]:
data_class = TextClasDataBunch.from_df('', train_df=df_trn, valid_df=df_val, test_df=df_tst, vocab=data_lm.vocab)
data_class.show_batch()

In [ ]:
learn_class.show_results(learn_class.data.test_ds)

In [ ]:
learn_class = text_classifier_learner(data_class, AWD_LSTM, drop_mult=0.5)
learn_class.load_encoder(RESULT_PATH/'encoder_lm.pkl')

In [ ]:
learn_class.lr_find()

In [ ]:
learn_class.recorder.plot()

In [ ]:
learn_class.fit_one_cycle(1, 1e-1, moms=(0.8,0.7))

In [ ]:
learn_class.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

In [ ]:
learn_class.unfreeze()

In [ ]:
learn_class.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

In [ ]:
from sklearn.metrics import accuracy_score

true_dict = {key: value for key, value in enumerate(learn_class.data.classes)}
preds = learn_class.get_preds(ds_type=DatasetType.Fix, with_loss=True, ordered=True)
preds = to_np(preds[0]).argmax(axis=1)
final_preds = np.vectorize(true_dict.get)(preds)
accuracy_score(df_trn['label'], final_preds)

In [ ]:
from sklearn.metrics import accuracy_score

true_dict = {key: value for key, value in enumerate(learn_class.data.classes)}
preds = learn_class.get_preds(ds_type=DatasetType.Valid, ordered=True, with_loss=True)
preds =to_np(preds[0]).argmax(axis=1)
final_preds = np.vectorize(true_dict.get)(preds)
accuracy_score(df_val['label'], final_preds)

In [ ]:
from sklearn.metrics import accuracy_score

true_dict = {key: value for key, value in enumerate(learn_class.data.classes)}
preds = learn_class.get_preds(ds_type=DatasetType.Test, ordered=True, with_loss=True)
preds = to_np(preds[0])
pred_df = pd.DataFrame(preds, columns = learn_class.data.classes)

In [ ]:
pred_df